# 데이터 불러오기
- 기간: 2019년1월 ~ 2020년4월

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import numpy as np
import unicodedata
import os
import time
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
from matplotlib import rcParams

data_dir = '/content/drive/My Drive/Colab Notebooks/프로젝트랩/data/'

#data = pd.read_csv(data_dir + 'data_19_0106.csv')
#data = pd.read_csv(data_dir + 'data_19_0712.csv')
#data = pd.read_csv(data_dir + 'data_20_0103.csv')
data = pd.read_csv(data_dir + '울산과학기술원_20.4월.txt', engine='python', encoding='euc-kr')

Mounted at /content/drive


# 업종 추가
- 사용 파일: column_categorize_bc_card.xlsx
- 기존의 업종중분류명 30개를 16개로 재분류
- 업종 예시: 음식점, 의료, 교통, 식료품, 오락 및 문화, 서비스, 의류 및 잡화, 자동차, 보험, 교육, 여행 및 숙박, 가구 및 가전, 기타, 대형 판매점, 종합소매, 전자상거래

In [ ]:
tmp = data

middle_category_money = tmp['업종중분류명'] == '유통업영리'
middle_category_not_money = tmp['업종중분류명'] == '유통업비영리'

large_category_life = tmp['업종대분류명'] == '생활'
large_category_shopping = tmp['업종대분류명'] == '쇼핑'

life_not_money_ = tmp[middle_category_not_money&large_category_life]
life_money_ = tmp[middle_category_money&large_category_life]
shopping_money_ = tmp[middle_category_money&large_category_shopping]

# 쇼핑 유통업영리 이용건수 총 합 -> 대형 판매에 들어가는 일반백화점, 대형할인점, 면세점만 
split_shopping_money_1 = shopping_money_['업종소분류명'].str.contains('대형할인점|일반백화점|면세점')
shopping_money_1 = shopping_money_[split_shopping_money_1]
shopping_money_1_sum = shopping_money_1.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

# 쇼핑 유통업영리 이용건수 총 합 -> 전자상거래에 들어가는 통신판매, 홈쇼핑, PG상품권 
split_shopping_money_2 = shopping_money_['업종소분류명'].str.contains('통신판매|홈쇼핑|PG상품권')
shopping_money_2 = shopping_money_[split_shopping_money_2]
shopping_money_2_sum = shopping_money_1.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

# 생활 유통업비영리 이용건수 총 합 -> 종합소매에 들어가는 all
life_not_money_sum = life_not_money_.groupby(['업종소분류명'],as_index = False).sum()['이용건수']


# 생활 유통업영리 이용건수 총합 -> 종합소매에 들어가는 슈퍼마켓 편의점, 연쇄점,복지, 매점, 농축수산가공품
split_life_money_1 = life_money_['업종소분류명'].str.contains('슈퍼마켓|편의점|연쇄점|복지매점|농축수산가공품')
life_money_1 = life_money_[split_life_money_1]
life_money_1_sum = life_money_1.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

# 생활 유통업영리 이용건수 총합 -> 전자상거래에 들어가는 인터넷종합Mall|상품권|인터넷PG|복지매점|전자상거래상품권
split_life_money_2 = life_money_['업종소분류명'].str.contains('인터넷종합Mall|상품권|인터넷PG|전자상거래상품권|인터넷Mall')
life_money_2 = life_money_[split_life_money_2]
life_money_2_sum = life_money_2.groupby(['업종소분류명'],as_index = False).sum()['이용건수']

tmp_data_1904 = tmp[['업종대분류명','업종중분류명','업종소분류명','이용건수']]
tmp_df = tmp_data_1904.groupby(['업종대분류명', '업종중분류명','업종소분류명'], as_index=False).sum()
tmp_df.columns = ['Large','Middle','Small','Count']
tmp_df = pd.DataFrame(tmp_df)
'''Large_sum = tmp_df.groupby(['Large'],as_index = False).sum()['Count']
for i in range(len(Large_sum)):
    a = {tmp_df['Large'].unique()[i] : Large_sum[i]}
    print(a)'''


classified_df =  pd.read_excel(data_dir + 'column_categorize_bc_card.xlsx')
concat_name = pd.DataFrame(classified_df['Large'] + '-' + classified_df['Middle'])
concat_name = concat_name.dropna()
new_classified = pd.concat([concat_name,classified_df['분류'],classified_df['total count']],axis=1)
new_classified.columns = ['category','num','total count']
new_category_name = list(classified_df['종목'][:16])
new_category_name += '대형판매', '종합소매', '전자상거래'
num_name = list(classified_df['기준'][:16])


for i in range(len(new_classified)):
    if new_classified['num'][i] == 'x':
        continue
    elif int(new_classified['num'][i]) == 1:
        new_classified.at[i,'new_category_name'] = new_category_name[0]
        
    elif int(new_classified['num'][i]) == 2:
        new_classified.at[i,'new_category_name'] = new_category_name[1]
        
    elif int(new_classified['num'][i]) == 3:
        new_classified.at[i,'new_category_name'] = new_category_name[2]
        
    elif int(new_classified['num'][i]) == 4:
        new_classified.at[i,'new_category_name'] = new_category_name[3]
    
    elif int(new_classified['num'][i]) == 5:
        new_classified.at[i,'new_category_name'] = new_category_name[4]
    
    elif int(new_classified['num'][i]) == 6:
        new_classified.at[i,'new_category_name'] = new_category_name[5]
        
    elif int(new_classified['num'][i]) == 7:
        new_classified.at[i,'new_category_name'] = new_category_name[6]
        
    elif int(new_classified['num'][i]) == 8:
        new_classified.at[i,'new_category_name'] = new_category_name[7]
        
    elif int(new_classified['num'][i]) == 9:
        new_classified.at[i,'new_category_name'] = new_category_name[8]
        
    elif int(new_classified['num'][i]) == 10:
        new_classified.at[i,'new_category_name'] = new_category_name[9]    
        
    elif int(new_classified['num'][i]) == 11:
        new_classified.at[i,'new_category_name'] = new_category_name[10]
        
    elif int(new_classified['num'][i]) == 12:
        new_classified.at[i,'new_category_name'] = new_category_name[11]
        
    elif int(new_classified['num'][i]) == 13:
        new_classified.at[i,'new_category_name'] = new_category_name[12]
    
    else:
        continue


added_category = {
    'category' : ['일반백화점','슈퍼마켓','홈쇼핑'],
    'num' : ['x','x','x'],
    'total count' : [shopping_money_1_sum.sum(), 
                     life_not_money_sum.sum()+life_money_1_sum.sum(),
                    life_money_2_sum.sum()+shopping_money_2_sum.sum()],
    'new_category_name' : ['대형 판매', '종합소매', '전자상거래']
}
added_df = pd.DataFrame(added_category)


new_classified = new_classified.dropna()
new_classified = pd.concat([new_classified,added_df],axis=0)
new_classified = new_classified.drop(['num'],axis=1)
new_classified = new_classified.reset_index()
new_classified = new_classified.drop(['index'],axis=1)


### str -> int mapping

def change_category (df):
    new_list = []
    a = list(df.category.unique())
    for i in range(len(a)):
        new_list.append(i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic

def change_upjong (df):
    new_list = []
    a = list(df.업종소분류명.unique())
    for i in range(len(a)):
        new_list.append(100+i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic



from tqdm import tqdm

z = tmp
z['category'] = z['업종대분류명']+ '-' + z['업종중분류명']
z = z[['category','업종소분류명']]
z = z.to_numpy()

change_category = change_category(tmp)
change_upjong = change_upjong(tmp)

for i in tqdm(range(len(z)),'1번째'):
    z[i,0] = change_category[z[i,0]]
    
for i in tqdm(range(len(z)),'2번째'):
    z[i,1] = change_upjong[z[i,1]]

2번째: 100%|██████████| 5469328/5469328 [00:03<00:00, 1601692.93it/s]


In [ ]:
number_one = change_category['생활-유통업영리']
number_two = change_category['생활-유통업비영리']
number_three = change_category['쇼핑-유통업영리']
print(number_one, number_two, number_three)

8 7 30


In [ ]:
for i in range(len(new_classified)):
    try:
        new_classified.category[i] = change_category[new_classified.category[i]]
    except:
        continue


new_classified = new_classified.drop(['total count'],axis=1)
new_classified = new_classified.to_numpy()
#new_classified


l_1 = []
l_2 = []
s_1 = []
s_2 = []

for i in list(life_money_1.업종소분류명.unique()):
    l_1.append(change_upjong[i])
    
# life money 2 4check
for i in list(life_money_2.업종소분류명.unique()):
    l_2.append(change_upjong[i])
    
for i in list(shopping_money_1.업종소분류명.unique()):
    s_1.append(change_upjong[i])
    
for i in list(shopping_money_2.업종소분류명.unique()):
    s_2.append(change_upjong[i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
tmp_list = []

for i in tqdm((range(len(z))),'진행도'):
    for k in range(len(new_classified)):   
        if z[i,0] == new_classified[k,0]:
            tmp_list.append(new_classified[k,1])
            
    if z[i,0] == number_one:    ############# 생활 유통업영리
        if z[i,1] in l_1:     
            tmp_list.append('종합소매')
            continue
        
        elif z[i,1] in l_2:   
            tmp_list.append('전자상거래')
            continue

        else:
            tmp_list.append('예외처리')
            continue

    elif z[i,0] == number_two:   ############## 생활 유통업비영리
        tmp_list.append('종합소매')
        continue

    elif z[i,0] == number_three:  ############## 쇼핑 유통업영리
        if  z[i,1] in s_1:
            tmp_list.append('대형 판매')
            continue

        elif z[i,1] in s_2:
            tmp_list.append('전자상거래')
            continue

        else:
            tmp_list.append('예외처리')
            continue

(unique, counts) = np.unique(tmp_list, return_counts=True)
frequencies = np.asarray((unique, counts)).T
frequencies

진행도: 100%|██████████| 5469328/5469328 [01:31<00:00, 59968.03it/s]


array([['가구 및 전자 제품', '55616'],
       ['교육', '171912'],
       ['교통', '27730'],
       ['기타', '114946'],
       ['대형 판매', '164292'],
       ['보험', '3457'],
       ['서비스', '358641'],
       ['식료품', '645928'],
       ['여행 및 숙박', '384062'],
       ['예외처리', '8902'],
       ['오락 및 문화', '358480'],
       ['음식점', '1024129'],
       ['의료', '623697'],
       ['의류 및 잡화', '214454'],
       ['자동차', '393068'],
       ['전자상거래', '20724'],
       ['종합소매', '899290']], dtype='<U21')

In [ ]:
""" 17개 맞는지 확인 """ 

category_df = pd.DataFrame(data=tmp_list,columns=['new_category'])
print(len(category_df.new_category.unique()))

In [ ]:
df = tmp.copy()
df['업종'] = tmp_list
df.isnull().sum()

기준년월        0
고객유형        0
가맹점소재지1     0
가맹점소재지2     0
가맹점소재지3     0
업종대분류명      0
업종중분류명      0
업종소분류명      0
성별          0
연령대별        0
가구생애주기별     0
연평균소득추정     0
이용금액        0
이용건수        0
category    0
업종          0
dtype: int64

# 전처리
- 필요없는 정보, 결측치, 이상치 제거

## 필요없는 정보 삭제

In [ ]:
""" 필요없는 col 삭제 """ 
try:
    del df['고객유형'] # 필요없으므로 삭제 
    del df['가구생애주기별'] # 연령으로 설명되므로 삭제
except:
    pass

""" 연령대별 col: '20세 미만' + '20대' 합치기 (='20대 이하') """
df['연령대별'] = df['연령대별'].replace({'20세 미만': '20대 이하', '20대': '20대 이하'})


## 결측치 및 'x'값 제거

In [ ]:
temp1 = df.shape

""" row 삭제: 연령대/가구/소득 x값 """ 
df.drop(df[df.연령대별 == 'x'].index, inplace=True)
df.drop(df[df.연평균소득추정 == 'x'].index, inplace=True)
#4월 txt에는 연령대별과 가구생애 주기별은 'x' 값 없음.

""" 결측치 제거 """
if df.isnull().sum().sum() != 0:
  df.dropna(axis=1,how='any')

temp2 = df.shape
temp3 = temp1[0]-temp2[0]  

record1_1 = '데이터 shape 변화: {0} -> {1}'.format(temp1, temp2)
record1_2 = '삭제된 row 수: {0}개, 전체의 {1}%'.format(temp3, round(temp3/temp1[0]*100),1)

print('결측치 제거')
print(record1_1)
print(record1_2)

결측치 제거
데이터 shape 변화: (5469328, 14) -> (5336937, 14)
삭제된 row 수: 132391개, 전체의 2%


## 이상치 제거
- 업종소분류 기준 [이용금액/이용건수] 이상치 5%까지만 제거

In [ ]:
def outliers_iqr(data, m):
  q1=data.quantile(0.25)
  q3=data.quantile(0.75)
  iqr=q3-q1
  lower_bound = q1 - (iqr*m)
  upper_bound = q3 + (iqr*m)
  search_df = data[(data < lower_bound) 
  | (data > (upper_bound))]
  return search_df

temp1 = df.shape

a=[]
type_list = df['업종소분류명'].unique()

for i in tqdm(range(len(type_list)),'진행도'):
  y_data=df[df['업종소분류명']==type_list[i]]['이용금액']
  x_data=df[df['업종소분류명']==type_list[i]]['이용건수']
  money_unit=y_data/x_data  
  per=outliers_iqr(money_unit,1.5).size/money_unit.size*100 #전체데이터에서 이상치의 퍼센트 확인
  j=0
  while per > 5 :
    per = outliers_iqr(money_unit,1.5+0.5*j).size/money_unit.size*100
    j+=1
  df = df.drop(outliers_iqr(money_unit,1.5+0.5*j).index, axis=0) # 이상치 값있는 행 제거   

temp2 = df.shape
temp3 = temp1[0]-temp2[0]  

record2_1 = '데이터 shape 변화: {0} -> {1}'.format(temp1, temp2)
record2_2 = '삭제된 row 수: {0}개, 전체의 {1}%'.format(temp3, round(temp3/temp1[0]*100),1)

print('이상치 제거')
print(record2_1)
print(record2_2)

진행도: 100%|██████████| 234/234 [06:51<00:00,  1.76s/it]

이상치 제거
데이터 shape 변화: (5336937, 14) -> (5136625, 14)
삭제된 row 수: 200312개, 전체의 4%


# 클러스터링 데이터셋 생성 (고객 그룹화)
- col: 업종
  - 음식점, 의료, 교통, 식료품, 오락 및 문화, 서비스, 의류 및 잡화, 자동차, 보험, 교육, 여행 및 숙박, 가구 및 가전, 기타, 대형 판매점, 종합소매, 전자상거래
- row: 고객 그룹
  - 지역, 성별, 연령대, 가구생애주기, 연평균소득 
- value: 이용금액(%)

In [ ]:
""" 고객 그룹별 전체 이용금액 """

grouped = df.groupby(['가맹점소재지2', '성별','연령대별', '연평균소득추정']) #경우의수: 
grouped_sum = grouped['이용금액'].sum()
df_spend_total = pd.DataFrame(grouped_sum)
df_spend_total

""" 고객 그룹별 업종 이용금액 """

grouped = df.groupby(['업종', '가맹점소재지2','성별','연령대별', '연평균소득추정'])
grouped_sum = grouped['이용금액'].sum()
df_spend_part = pd.DataFrame(grouped_sum)
df_spend_part

############################시간단축###########################################
# 기존 코드: 10만개 4분
# 수정 코드: 500만개 8분33초
# numpy로 수정해보기 ,,,,

""" str -> int mapping """
""" index: 가맹점소재지2, 성별, 연령, 소득 """ 
""" col 값을 바꾸는 게 아니라 index 값을 바꾸는거 """

def change_region (df):
    new_list = []
    a = list(df.index.get_level_values('가맹점소재지2').unique())
    for i in range(len(a)):
        new_list.append(i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic

def change_sex (df):
    new_list = []
    a = list(df.index.get_level_values('성별').unique())
    for i in range(len(a)):
        new_list.append(300+i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic

def change_age (df):
    new_list = []
    a = list(df.index.get_level_values('연령대별').unique())
    for i in range(len(a)):
        new_list.append(400+i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic

def change_income (df):
    new_list = []
    a = list(df.index.get_level_values('연평균소득추정').unique())
    for i in range(len(a)):
        new_list.append(500+i)
    dic = {key:value for key,value in zip(a,new_list)}
    return dic

from tqdm import tqdm

#z = df_spend_part
#z = z[['가맹점소재지2','성별', '연령대별', '연평균소득추정']]
#z = z.to_numpy()

change_region = change_region(df_spend_part)
change_sex = change_sex(df_spend_part)
change_age = change_age(df_spend_part)
change_income = change_income(df_spend_part)

df_spend_part.rename(index = change_region, inplace=True)
df_spend_part.rename(index = change_sex, inplace=True)
df_spend_part.rename(index = change_age, inplace=True)
df_spend_part.rename(index = change_income, inplace=True)

df_spend_total.rename(index = change_region, inplace=True)
df_spend_total.rename(index = change_sex, inplace=True)
df_spend_total.rename(index = change_age, inplace=True)
df_spend_total.rename(index = change_income, inplace=True)


###############################################################################

""" 이용금액 % 생성 """

for index,row in tqdm(df_spend_part.iterrows()):
    
    sort_list = list(index)[1:] # e.g., 강릉시 남성 20대 B1
    region, sex, age, income = sort_list[0], sort_list[1], sort_list[2], sort_list[3]
    spend_total = df_spend_total.loc[   (df_spend_total.index.get_level_values('가맹점소재지2') == region)
                                      & (df_spend_total.index.get_level_values('성별') == sex) 
                                      & (df_spend_total.index.get_level_values('연령대별') == age)
                                      & (df_spend_total.index.get_level_values('연평균소득추정') == income)]['이용금액']
    
    spend_total = int(spend_total)    
    df_spend_part.at[index, '총 이용금액'] = spend_total
    
df_spend_part['이용금액%'] = df_spend_part['이용금액'] / df_spend_part['총 이용금액'] * 100
df_spend_part['이용금액%'] = df_spend_part['이용금액%']        # round(df_spend_part['이용금액%'], 1)
df_spend_part

""" 데이터셋 형태로 만들기 """

df_final = df_spend_part.copy()
df_final.drop(['이용금액', '총 이용금액'], axis=1, inplace=True)
df_final = df_final.reset_index(level=[0])
df_final = pd.pivot_table(df_final, values='이용금액%', columns='업종', 
                                   index=['가맹점소재지2', '성별', '연령대별', '연평균소득추정'])
df_final = df_final.fillna(0) # NaN -> 0
df_final["합계%"] = df_final.sum(axis=1)

""" reverse dict """ 

change_region = {value : key for (key, value) in change_region.items()}
change_sex = {value : key for (key, value) in change_sex.items()}
change_age = {value : key for (key, value) in change_age.items()}
change_income = {value : key for (key, value) in change_income.items()}
df_final.rename(index = change_region, inplace=True)
df_final.rename(index = change_sex, inplace=True)
df_final.rename(index = change_age, inplace=True)
df_final.rename(index = change_income, inplace=True)

print(df_final.shape)

155577it [06:24, 404.62it/s]


(16264, 18)


In [ ]:
df_final # 데이터셋 합계%가 100%인지 확인

업종                         가구 및 전자 제품        교육   교통  ...  전자상거래       종합소매    합계%
가맹점소재지2 성별 연령대별   연평균소득추정                             ...                         
가평군     남성 20대 이하 B3         0.015728  1.063637  0.0  ...    0.0  29.971287  100.0
                  B4         0.000000  0.475363  0.0  ...    0.0  29.065658  100.0
                  B5         0.000000  0.405091  0.0  ...    0.0  24.663780  100.0
                  B6         0.000000  0.000000  0.0  ...    0.0  30.162521  100.0
                  B2         0.000000  1.413551  0.0  ...    0.0  31.744414  100.0
...                               ...       ...  ...  ...    ...        ...    ...
청송군     여성 60대 이상 B3         0.000000  0.000000  0.0  ...    0.0  28.201648  100.0
                  B4         0.000000  0.000000  0.0  ...    0.0  42.769951  100.0
                  B5         0.000000  0.000000  0.0  ...    0.0   0.000000  100.0
                  B2         0.000000  0.000000  0.0  ...    0.0  25.766032  100.0
                  B1         0.000000  0.000000  0.0  ...    0.0  12.245734  100.0

[16264 rows x 18 columns]